# Vehicle CAN analysis

## 1. Objectives
Dynamical systems model the complex relationships between quantities that evolve over time. 
Formally, dynamical systems analyze, predict, and interpret systems of differential equations or iterative mappings that describe the state development of a system. This formulation covers a wide range of phenomena in classical mechanical systems, electrical circuits, turbulent fluids, climate science, finance, ecology, social systems, neuroscience, epidemiology, and nearly every other system that evolves over time.

Poincare's work on planets' chaotic motion started modern dynamical systems. It is based on classical mechanics and represents centuries of mathematical modeling, starting with Newton and Leibniz. Dynamical systems have fascinated brilliant brains for millennia and been applied to various fields and difficult issues. Dynamical systems connects linear algebra, differential equations, topology, numerical analysis, and geometry. Nearly all technical, physical, and life sciences use dynamical systems to model and analyze systems.

Classical controls and dynamics systems are expressed in terms:
\begin{equation*}
   	\dot{x} = Ax+Bu,
\end{equation*}

\begin{equation*}
	y= Cx
\end{equation*}

where:
* X: a state vector or state variable of the system. It typically represents the internal state or variables of a dynamic system that evolves over time.
* X_dot: The derivative of the state vector X with respect to time, indicating how the state variables change over time.
* A: A matrix that describes the dynamics or evolution of the state variables. It defines how the state variables interact and change over time in the absence of any inputs or disturbances.
* B: A matrix that represents the control input vector or the effect of external inputs on the state variables. It shows how the system responds to control inputs or external influences.
* u: The control input vector or the input applied to the system to affect its behavior or state evolution. It can be considered as the driving force or external signal acting on the system.
* y: The output vector that represents the measurable or observable variables of the system. It is derived from the state variables or directly observed from the system.
* C: A matrix that relates the state variables to the output variables. It defines how the state variables contribute to the output measurements or observations.

Data-driven techniques are replacing analytical derivations and first principles models in dynamical systems. Big data and machine learning are changing how scientists and engineers analyze dynamical systems. Climate science, finance, epidemiology, and neuroscience problems have ample data but no controlling equations. Researchers are increasingly using data-driven analysis in classical domains like optics and turbulence, where governing equations exist. 

The Machine Learning techniques provide a new set of tools that can help to understand the system dynamics, thus enabling developing the controls systems without **without knowing the full system dynamics**.

In this exercise, I'd like to experiment the modern data-driven controls and dynamical systems approach **without knowing the full system dynamics**.
1. Using Regression technique, we can predict the desired outputs Y (i.e, Vehicle Speed) 
2. Using Classifiers, we can predict the control inputs U (i,e. Brake Voltage)
3. Using Time Series anomaly detection to predictively maintain the system

References: http://databookuw.com/databook.pdf 

## 2. Data Understanding


The dataset is the Vehicle CAN data that is collected directly from the CAN bus inside vehicles, representing the measurements y and inputs u in the system.

The dataset contains 20Hz sampled CAN bus data from a passenger vehicle, e.g. 
* WheelSpeed FL (speed of the front left wheel), 
* SteerAngle (steering wheel angle), 
* Role, Pitch, and accelerometer values per direction.

Unfortunately, not much further details on the measurement units or how they were collected are given.

In contrast to the dataset published at https://zenodo.org/record/2658168#.XMw2m6JS9PY we now have GPS data from the vehicle (see signals 'Latitude_Vehicle' and 'Longitude_Vehicle' in h5 group 'Math') and GPS data from the IMU device (see signals 'Latitude_IMU', 'Longitude_IMU' and 'Time_IMU' in h5 group 'Math') included. However, as it was exported with single_precision, therefore we lost some precision for those GPS values.

References: https://zenodo.org/record/2661316 

In [ ]:
import h5py # to read the hdf file

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
import time
import warnings
import string
import re

### Load vehicle data

In [ ]:
driver_1 = h5py.File('20181116_Driver1_Trip3.hdf', 'r')

Both files contain multiple subgroups of data, one of which is the aformentioned CAN bus:

In [ ]:
list(driver_1.keys())

### Turn time series data into tables

The CAN bus data comes in serialized form - written out in series in a nested format.

To handle the CAN bus data more efficiently we'll turn it into tables that are easier to inspect and handle.

In [ ]:
data_driver_1 = {}

for channel_name, channel_data in driver_1['CAN'].items():
  data_driver_1[channel_name] = channel_data[:, 0]

df_raw = pd.DataFrame(
    data=data_driver_1,
    index=channel_data[:, 1]
)
df_raw = df_raw.loc[:, df_raw.nunique() > 1]

In [ ]:
df_raw.info()

The tabular data for driver 1 looks as follows - it holds 30746 measured time points in 23 channels that we deem relevant

In [ ]:
df_raw.head()

In [ ]:
df_raw.describe()

### Data description

* User Controls Inputs
  * AccPedal
  * BrkVoltage
  * SteerAngle1
* Measurements
  * AmbientTemperature
  * BoostPressure
  * ENG_Trq_DMD
  * ENG_Trq_ZWR
  * ENG_Trq_m_ex
  * EngineSpeed_CAN
  * EngineTemperature
  * Engine_02_BZ
  * Engine_02_CHK
  * OilTemperature1
  * SCS_01_BZ
  * SCS_01_CHK
  * Trq_FrictionLoss
  * Trq_Indicated
  * WheelSpeed_FL
  * WheelSpeed_FR
  * WheelSpeed_RL
  * WheelSpeed_RR
  * Yawrate1
* Vehicle Output under controls
  * VehicleSpeed
  * SteerAngle1

Some of the abbreviations are not fully understanding due to lack of information. We could gather them via contacting the source of the data. However it should not block the progress here.

## 3. Data Preparation

In [ ]:
def data_sanity_check(df):
    #print(df.describe())
    print('===Null values per columns: ')
    print(df.isnull().sum())
    print(f'===Duplicates found: {df.duplicated().sum()}')
    df = df.drop_duplicates()
    print(f'===Duplicates removed. Duplicates found: {df.duplicated().sum()}')

    return df

In [ ]:
df_raw = data_sanity_check(df_raw)

In [ ]:
#Explore unique data in columns
df_unique = df_raw.nunique().to_frame().reset_index()
df_unique.columns = ['Variable','Unique counts']
print(df_unique)

### Visualize numeric data

NOTE: This line is only for exploratory, since it is time consuming to execute. It is good to run it once to give us an idea how the dataset looks like.

In [ ]:
fig = px.line(df_raw, title='time series of Vehicle CAN data')
fig.show()

The EngineSpeed_CAN skews the data scale. Lets normalize the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
 
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_raw)
scaled_df_raw = pd.DataFrame(scaled_data,
                         columns=df_raw.columns)
scaled_df_raw.head()

In [ ]:
fig = px.line(scaled_df_raw, title='Time series of Vehicle CAN data')
fig.show()

**Observation** \
The data looks very noisy, need to drop some noisy data. 


In [ ]:
scaled_df_raw.columns
## Drop noisy data columns
scaled_df1 = scaled_df_raw.drop(columns=['SCS_01_BZ', 'SCS_01_CHK', 'Engine_02_BZ', 'Engine_02_CHK'], axis=1)
df1 = df_raw.drop(columns=['SCS_01_BZ', 'SCS_01_CHK', 'Engine_02_BZ', 'Engine_02_CHK'], axis=1)
fig = px.line(scaled_df1, title='Time series of Vehicle CAN data, take-1 clean')
fig.show()

**Observation** \
The data looks much better and less noisy!

Let's examine some critical info of the dataset such as Vehicle speed, temperature, etc


In [ ]:
fig = px.line(scaled_df1, y= ['VehicleSpeed', 'BrkVoltage'], title='Time series of braking voltage vs Vehicle Speed')
fig.show()

In [ ]:
fig = px.line(scaled_df1, y= ['VehicleSpeed', 'SteerAngle1', 'AccPedal'], title='Time series of acceleration pedal and steering angle vs Vehicle Speed')
fig.show()

**Observation** \
Brake Voltage, Acceleration Pedal, and Steering Angle are 3 critical control inputs coming from the human users that determines the speed of the vehicle. We will build the model to predict these control inputs later on.


In [ ]:
fig = px.line(df1, 
              y= ['AirIntakeTemperature', 'AmbientTemperature', 'EngineTemperature', 'OilTemperature1', 
                  'VehicleSpeed'                          
                              ], title='Time series of All temperature related vs Vehicle Speed')
fig.show()

**Observation** \
Temperature measurements are responding to the vehicle speed, however due to the temperature dynamics, the responds exhibit some delays that become hard for controls scheme.

### Visualize data distributions

In [ ]:
# Visualize data distributions
# Numeric variables
cols_numeric = df1.select_dtypes(exclude=['object']).columns

for col in cols_numeric:
    #sns.histplot(df_raw[col], kde=True)
    sns.histplot(data =df1, x = col, kde=True) #hue='y', kde=True)
    plt.title(f"Distribution of {col}")
    colname = col.replace(".", "_")
    plt.savefig(f"results/EDA_{colname}")
    plt.show()

## Data Modeling I - Regression model - predict Vehicle Speed

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, balanced_accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, RocCurveDisplay , precision_recall_curve, PrecisionRecallDisplay

# Import PCA
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance

# Ignore warnings
warnings.filterwarnings('ignore')

### Regression - Feature Engineeering

In [ ]:
#Describe data and visualize it
corr = df1.corr()

plt.figure(figsize=(10,10))
ax = sns.heatmap(
    corr[abs(corr) >=.5], 
    #corr,
    vmin=-1, vmax=1, center=0,
    cmap="Reds",
    square=False,
    annot=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

plt.title('Correlation among the features that are above 0.7 score')
plt.savefig("results/EDA_feature_corr")
plt.show()


**Observations** \
There are heavily correlated data between WheelSpeeds (FL, FR, RL, RR) and the torques. It is understandable as the vehicle speed is calculated directed from the linear model with those values.

#### Variance Inflation Factor check

Let's use VIF to methodogically reduce the correlations among the columns.
(references: https://en.wikipedia.org/wiki/Variance_inflation_factor)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
def vif(vars, data):
    # rules of thumbs if VIF > 5 then there is a strong multi-colinearity among the features
    vif_dict ={}
    for var in vars:
        not_var = [i for i in vars if i !=var]
        X, y = data[not_var], data[var]
        
        r_squared = LinearRegression().fit(X,y).score(X,y)
        #calc the VIF
        vif = 1/(1-r_squared)
        vif_dict[var] = vif

    return pd.DataFrame({"VIF": vif_dict})

In [ ]:
vif(df1.columns, df1).sort_values("VIF")

In [ ]:
# drop columns that have VIF scores greater than 5
df2 = df1.drop(columns=['Trq_FrictionLoss', 'AirIntakeTemperature',
                        'ENG_Trq_ZWR', 'ENG_Trq_DMD', 'OilTemperature1', 'WheelSpeed_RL',  'WheelSpeed_RR', 'WheelSpeed_FL', 'WheelSpeed_FR', 'Trq_Indicated',
                        'ENG_Trq_m_ex', 'BoostPressure'
                        ], axis=1)

In [ ]:
df2.columns

In [ ]:
sns.heatmap(df2.corr(), annot = True, fmt=".1f")

In [ ]:
vif(df2.columns, df1).sort_values("VIF")

**Observations**\
All VIFs are less than 5, so we are good to go!

### Regression - Train/Test

In [ ]:
# Split the dataset into training and testing sets
X = df2.drop(columns="VehicleSpeed")
y = df2["VehicleSpeed"]

# Performed the train/test split with 30% test data and random state of 42 for shuffling.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print the size of the training and testing sets
print("X_train size:", X_train.shape)
print("y_train size:", y_train.shape)
print("X_test size:", X_test.shape)
print("y_test size:", y_test.shape)

In [ ]:
# install missing packages
! pip install mlxtend

In [ ]:
import timeit # for model CPU performance https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time
import sys

from sklearn.linear_model import LinearRegression, HuberRegressor, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel
from sklearn.datasets import make_blobs

import joblib
sys.modules['sklearn.externals.joblib'] = joblib # for SFS

from sklearn import set_config
set_config(display="diagram")

In [ ]:
# collect all mse/ mae values for each model
df_RegressionModelPerformance  = pd.DataFrame(columns=['model', 'mse', 'r2', 'CPU time'])
df_RegressionModelPerformance.info()

#### Regression Model 1: Linear Regresion with all 12 features

In [ ]:
%%time
lr_all_features = LinearRegression(fit_intercept=False).fit(X_train, y_train)
y_pred =  lr_all_features.predict(X_test)
lr_mse = mean_squared_error( y_test, y_pred)
lr_r2 = r2_score(y_test, y_pred)

df_RegressionModelPerformance.loc[0] = ['lr_all_features', lr_mse, lr_r2, -1] 
print(f'LinearRegression_AllFeatures / MSE = {lr_mse}')
print(f'LinearRegression_AllFeatures / R2 = {lr_r2}')
print(f'LinearRegression_AllFeatures / Coef = {lr_all_features.coef_}')

In [ ]:
print(pd.Series(lr_all_features.coef_, index=list(X_train.columns)).sort_values(ascending=True))

**Observation**

The LR indicates that Brake Voltage has the most impact of the Vehicle Speed, without other significant measurements! These are quite impressive results!


In [ ]:
# manually record CPU time
df_RegressionModelPerformance.loc[0,'CPU time'] = 3.99
df_RegressionModelPerformance.head(5)


#### Regression Model 2: Huber Regression with all 12 features

Huber regression with Huber loss is a composite of both MSE and MAE that plays a critical role. A higher loss results in the quadratic equation transforming into a linear equation. If the error is smaller than the cut-off (epsilon), MSE is used. Otherwise, MAE is used. Huber regression handles the outliers judiciously.


In [ ]:
%%time
huber_all_features = HuberRegressor(fit_intercept=False).fit(X_train, y_train)
y_pred =  huber_all_features.predict(X_test)
huber_mse = mean_squared_error(y_test, y_pred)
huber_r2 = r2_score(y_test, y_pred)

df_RegressionModelPerformance.loc[1] = ['huber_all_features', huber_mse, huber_r2, -1] 
print(f'HuberRegression_AllFeatures / MSE = {huber_mse}')
print(f'HuberRegression_AllFeatures / R2 = {huber_r2}')
print(f'HuberRegression_AllFeatures / Coef = {huber_all_features.coef_}')

In [ ]:
print(pd.Series(huber_all_features.coef_, index=list(X_test.columns)).sort_values(ascending=True))

In [ ]:
# manually record CPU time
df_RegressionModelPerformance.loc[1,'CPU time'] = 274
df_RegressionModelPerformance.head(5)


**Observations**

The Huber Regression gives a comparable prediction and the scores. In Huber, The most impacting feature is the Ambient temperature!

#### Regression Model 3: Polynomial Regression 2 degrees

In [ ]:
%%time
# poly transform
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

lr_poly_all_features = LinearRegression().fit(X_train_poly, y_train)

y_pred =  lr_poly_all_features.predict(X_test_poly)
poly_mse = mean_squared_error(y_test, y_pred)
poly_r2 = r2_score(y_test, y_pred)

df_RegressionModelPerformance.loc[2] = ['lr_poly_all_features', poly_mse, poly_r2, -1] 
print(f'PolyRegression_AllFeatures / MSE = {poly_mse}')
print(f'PolyRegression_AllFeatures / R2 = {poly_r2}')
print(f'PolyRegression_AllFeatures / Coef = {lr_poly_all_features.coef_}')

In [ ]:
# manually record CPU time
df_RegressionModelPerformance.loc[2,'CPU time'] = 19.5
df_RegressionModelPerformance.head(5)

#### Regression Model 4: Sequential Feature Selection with Polynomial Regression

In [ ]:
%%time

sfs_pipe = Pipeline([('poly_features', PolynomialFeatures(degree = 2, include_bias = False)),
                    ('selector', SequentialFeatureSelector(LinearRegression(), n_features_to_select=6)),
                    ('linreg', LinearRegression())])
sfs_pipe.fit(X_train, y_train)
y_pred = sfs_pipe.predict(X_test)
sfs_train_mse = mean_squared_error(y_train, sfs_pipe.predict(X_train))
sfs_test_mse = mean_squared_error(y_test, y_pred)
sfs_r2 = r2_score(y_test, y_pred)

sfs_pipe

In [ ]:
#sfs_coefs = sfs_pipe.named_steps['selector'].coeff_
df_RegressionModelPerformance.loc[3] = ['sfs', sfs_test_mse, sfs_r2, -1] 
sfs_feature_names= sfs_pipe.named_steps['selector'].get_feature_names_out()
print(f'SFS / Features = {sfs_feature_names}')
print(f'SFS / MSE = {sfs_test_mse}')
print(f'SFS / R2 = {sfs_r2}')

In [ ]:
# manually record CPU time
df_RegressionModelPerformance.loc[3,'CPU time'] = 2_700
df_RegressionModelPerformance.head(5)

#### Regression Model 5: Lasso Regularization with Polynomial Regression

In [ ]:
%%time
lasso_pipe = Pipeline([('polyfeatures', PolynomialFeatures(degree = 2, include_bias = False)),
                      ('scaler', StandardScaler()),
                     ('lasso', Lasso(random_state = 42))])
lasso_pipe.fit(X_train, y_train)
y_pred = lasso_pipe.predict(X_test)

lasso_coefs = lasso_pipe.named_steps['lasso'].coef_
lasso_train_mse = mean_squared_error(y_train, lasso_pipe.predict(X_train))
lasso_test_mse = mean_squared_error(y_test, lasso_pipe.predict(X_test))
lasso_r2 = r2_score(y_test, y_pred)

df_RegressionModelPerformance.loc[4] = ['lasso', lasso_test_mse, lasso_r2, -1] 
lasso_feature_names = lasso_pipe.named_steps['polyfeatures'].get_feature_names_out()
print(f'Lasso / Features = {lasso_feature_names}')
print(f'Lasso / MSE = {lasso_test_mse}')
print(f'Lasso / R2 = {lasso_r2}')

In [ ]:
# manually record CPU time
df_RegressionModelPerformance.loc[4,'CPU time'] = 103_300
df_RegressionModelPerformance.head(5)

#### Regression Model 6: Ridge regularization with GridSearch

In [ ]:
%%time

ridge_param_dict = {'ridge__alpha': np.logspace(0, 10, 50)}
ridge_pipe = Pipeline([('scaler', StandardScaler()), 
                      ('ridge', Ridge())])
ridge_grid = GridSearchCV(ridge_pipe, param_grid=ridge_param_dict,scoring = 'neg_root_mean_squared_error')
ridge_grid.fit(X_train, y_train)
ridge_train_preds = ridge_grid.predict(X_train)
ridge_test_preds = ridge_grid.predict(X_test)
ridge_train_mse = mean_squared_error(y_train, ridge_train_preds)
ridge_test_mse = mean_squared_error(y_test, ridge_test_preds)
ridge_r2 = r2_score(y_test, y_pred)

ridge_pipe

df_RegressionModelPerformance.loc[5] = ['ridge', ridge_test_mse, ridge_r2, -1] 
print(f'Ridge / MSE = {ridge_test_mse}')
print(f'Ridge / R2 = {ridge_r2}')

In [ ]:
# manually record CPU time
df_RegressionModelPerformance.loc[5,'CPU time'] = 1_520
df_RegressionModelPerformance.head(5)

#### Regression Model 7: Ridge and Transformed Target Regressor with GridSearchCV

As recall from the Price plot distribution, the Price is not normally distributed and let's try using the transform target regressor to help improve the prediction.

In [ ]:
from sklearn.compose import TransformedTargetRegressor

In [ ]:
%%time
ttr = TransformedTargetRegressor(regressor = Ridge(), transformer=MinMaxScaler())
ss = MinMaxScaler()

ridge_tts_pipe = Pipeline([('polyfeatures', PolynomialFeatures(degree = 2, include_bias = False)),
                 ('ttregressor', ttr)])

ridge_tts_pipe.fit(X_train, y_train)
ridge_tts_pipe.score(X_test, y_test)

param_grid_ridge_tts = {'ttregressor__regressor__alpha':[True,False]}
grid_ridge_tts = GridSearchCV(ridge_tts_pipe,param_grid=param_grid_ridge_tts, cv=None)
grid_ridge_tts.fit(X_train, y_train)
print("best alpha: ", grid_ridge_tts.best_score_)

ridge_tts_train_preds = grid_ridge_tts.predict(X_train)
ridge_tts_test_preds = grid_ridge_tts.predict(X_test)
ridge_tts_train_mse = mean_squared_error(y_train, ridge_tts_train_preds)
ridge_tts_test_mse = mean_squared_error(y_test, ridge_tts_test_preds)
ridge_tts_r2 = r2_score(y_test, ridge_tts_test_preds)

ridge_tts_pipe

df_RegressionModelPerformance.loc[6] = ['ridge_tts', ridge_tts_test_mse, ridge_tts_r2, -1] 
print(f'Ridge / MSE = {ridge_tts_test_mse}')
print(f'Ridge / R2 = {ridge_tts_r2}')

In [ ]:
# manually record CPU time
df_RegressionModelPerformance.loc[6,'CPU time'] = 272
df_RegressionModelPerformance.head(10)

#### Regression Model 9: Deep NN

In [75]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [ ]:
model1 = Sequential()
model1.add(Dense(50, activation = 'relu'))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss = 'bce', metrics = ['acc'])
history1 = model1.fit(X_train, y_train, validation_data = (X_test, y_test),
                     epochs = 10, verbose = 0)


## Data Modeling II - Classifiers - detect Controls Inputs

### Classifiers - Classify the Controls Inputs

In [ ]:
df3 =[]
df3 = df2
df3.info()

Categorize the Brake Voltage

In [ ]:
df3['BrkVoltage'] = df3['BrkVoltage'].astype(int)
df3['BrkVoltage']
fig = px.line(df3['BrkVoltage'], title='Time series of Vehicle CAN data - Brake Voltage')
fig.show()

**Classify the Steering Angles** \
Calculate the diffference in time series of the Steer angles:
* if the difference >0 then its a turn left input, 
* else it is a turn right input.

In [ ]:
df3 =df3.drop(columns= 'SteerAngle1')
df3['inpSteerDirection'] = df2['SteerAngle1'].diff().fillna(0)

# < 0 is turn left, > 0 is turn right
bins = [-1000, 0, 1000]
group_names = [ 'Left', 'Right'] 
df3['inpSteerDirection'] = pd.cut(df3['inpSteerDirection'], bins, labels=group_names)

fig = px.line(df3[['inpSteerDirection']], title='Time series of Vehicle CAN data - inpSteerDirection')
fig.show()

**Classify the Acceleration Pedal** \
Calculate the diffference in time series of the Acceleration Pedal:
* if the difference > 0 then the pedal is pressed, 
* else it is no pedal press

In [ ]:
df3 =df3.drop(columns= 'AccPedal')
df3['inpAccPedal'] = df2['AccPedal'].diff().fillna(0)

# < 0 is turn left, > 0 is turn right
bins = [-1000, 0, 1000]
group_names = [ 0, 1] 
df3['inpAccPedal'] = pd.cut(df3['inpAccPedal'], bins, labels=group_names)

fig = px.line(df3[['inpAccPedal']], title='Time series of Vehicle CAN data - AccPedal_inp')
fig.show()

### Classifiers - Label Encoding

In [ ]:
# Encode categorical columns
cols_le = ["inpSteerDirection", 'inpAccPedal']

# Label encoding
le = LabelEncoder()
encoded_le_df = df3[cols_le].apply(le.fit_transform)

# Combine encoded categorical variables with the original dataframe
df_encoded =[]
df_encoded = df3.drop(columns=cols_le)
df_encoded = pd.concat([df_encoded, encoded_le_df], axis=1, join='inner')

In [ ]:
df_encoded.info()

### Classifiers- Train/ Test

In [ ]:
# Split the dataset into training and testing sets
#cols = ["inpSteerDirection", "BrkVoltage","inpAccPedal" ]
cols = ["BrkVoltage"]
X = df_encoded.drop(columns=cols)
y = df_encoded[cols]

# Performed the train/test split with 30% test data and random state of 42 for shuffling.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print the size of the training and testing sets
print("X_train size:", X_train.shape)
print("y_train size:", y_train.shape)
print("X_test size:", X_test.shape)
print("y_test size:", y_test.shape)

#### Classifier - A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
%%time
# Instantiate a dummy classifier that always predicts the majority class
strategies = ['most_frequent', 'stratified', 'uniform']
  
test_scores = []

for s in strategies:
    
    dummy = DummyClassifier(strategy = s)
  
    # Fit the classifier on the training data
    dummy.fit(X_train, y_train)

    # Evaluate the classifier on the data
    accuracy_train = dummy.score(X_train, y_train)
    accuracy_test = dummy.score(X_test, y_test)
    # Print the baseline accuracy
    print(f"Strategy {s} --- Baseline accuracy train: {accuracy_train} --- Baseline accuracy test: {accuracy_test}")

#### Classifier Model 1 - A simple model 
Use Logistic Regression to build a basic model on your data.  

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
%%time

# Configure the search space 
logreg_params = {'solver': ['liblinear', 'lbfgs'], 'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10, 100], 'class_weight': [None, 'balanced']}
logreg_model=""

# Grid search CV for Logistic Regression
logreg_model = GridSearchCV(LogisticRegression(), logreg_params, cv=5)
logreg_model.fit(X_train, y_train)

logreg_best_params = logreg_model.best_params_
print(f"Logistic Regression best param: {logreg_best_params}")

# score the logistic regression model on the data
train_accuracy = logreg_model.score(X_train, y_train)
test_accuracy = logreg_model.score(X_test, y_test)
# Print the train  accuracy
print("Logistic Regression accuracy train: {:.5f}".format(train_accuracy))
print("Logistic Regression accuracy test: {:.5f}".format(test_accuracy))

#### Classifier Model Accuracy

In [ ]:
# accuracy of the model
print("Accuracy train: {:.5f}".format(train_accuracy))
print("Accuracy test: {:.5f}".format(test_accuracy))

# Make predictions on the test data
y_pred = logreg_model.predict(X_test)

# Calculate the precision, recall, and F1 score
precision = precision_score(y_test, y_pred, pos_label=1)
recall = recall_score(y_test, y_pred, pos_label=1)
f1 = f1_score(y_test, y_pred, pos_label=1)

# Print the performance metrics
print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))
print("F1 Score: {:.3f}".format(f1))

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a confusion matrix display
cmd = ConfusionMatrixDisplay(cm, display_labels=logreg_model.classes_)

# Plot the confusion matrix
cmd.plot(cmap= "Paired")
plt.title('Confusion Matrix for Logistic Regression model')

In [ ]:
# display the ROC curve

y_score = logreg_model.decision_function(X_test)

fpr, tpr, _ = roc_curve(y_test, y_score, pos_label=logreg_model.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
plt.title('Roc Curve Display for Logistic Regression model')

auc_score =roc_auc_score(y_test, logreg_model.predict_proba(X_test)[:,1])
print("Roc AUC Score: {:.5f}".format(auc_score))

#### Classifier Model Comparison

In [ ]:
# Initialize an empty DataFrame to store the performance
df_performance = pd.DataFrame(columns=['Model', 'Train Time', 'Train Accuracy', 'Test Accuracy'])

In [ ]:
# Model params for GridSearchCV
dec_tree_params = {'criterion': ['gini', 'entropy'], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
knn_params = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree'], 'leaf_size': [20, 30, 40]}
svm_params = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'degree': [2, 3], 'gamma': ['scale', 'auto'], 'class_weight': [None, 'balanced']}

##### !!! Precautions, the next cell can take 3 minutes to execute !!!

In [ ]:
# GridSearch for Decision Tree
dec_tree_grid = GridSearchCV(DecisionTreeClassifier(), dec_tree_params, cv=5)
dec_tree_grid.fit(X_train, y_train)
dec_tree_best_params = dec_tree_grid.best_params_
print("Decision Tree best params:", dec_tree_grid.best_params_)

##### !!! Precautions, the next cell can take 5 minutes to execute !!!

In [ ]:
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5)
knn_grid.fit(X_train, y_train)

In [ ]:
knn_best_params = knn_grid.best_params_
print("KNN best params:", knn_grid.best_params_)

##### !!! Precautions, the next cell can take up to 10 hours to execute !!!

In [ ]:
# GridSearch for SVM
svm_grid = GridSearchCV(SVC(), svm_params, cv=5)
svm_grid.fit(X_train, y_train)
svm_best_params = svm_grid.best_params_
print("SVM best params:", svm_grid.best_params_)

#### Observations
From the GridSearch, we are able to identify the best params for each model as below

In [ ]:
# Model parameters
# Logistic Regression best param: {'C': 10, 'class_weight': None, 'penalty': 'l2', 'solver': 'liblinear'}
log_reg_params = {'C': 10, 'class_weight': None, 'penalty': 'l2', 'solver': 'liblinear'}
# Decision Tree best params: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 0.1}
dec_tree_params = {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 0.1}
# KNN best params: {'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}
knn_params = {'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}
svm_params = {'C': 1.0, 'class_weight': 'balanced', 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}

In [ ]:
# Train and evaluate the models
classifiers = {
    "Logistic Regression": LogisticRegression(**log_reg_params),
    "Decision Trees": DecisionTreeClassifier(**dec_tree_params),
    "KNN": KNeighborsClassifier(**knn_params),
    "SVM": SVC(**svm_params)
}

# Custom function to evaluate metrics
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return {"Test Accuracy": accuracy, "Test f1": f1, "Test Precision": precision, "Test Recall": recall}

# Train and evaluate each classifier
metrics = {}
df_metrics = []
df_performance = []
df_performance = pd.DataFrame(columns=['Model', 'Train Time', 'Train Accuracy', 'Test Accuracy'])

for model, clf in classifiers.items():
    # Train the model and capture execution time
    start_time = time.time()

    clf.fit(X_train, y_train)

    end_time = time.time()
    train_time = end_time - start_time

    # Accuracy on train and test data
    train_accuracy = clf.score(X_train, y_train)
    test_accuracy = clf.score(X_test, y_test)
    
    # model performance
    df_model_performance = pd.DataFrame({'Model': [model], 'Train Time': [train_time], 'Train Accuracy': [train_accuracy], 'Test Accuracy': [test_accuracy]})

    # Compute all other metrics
    y_pred = clf.predict(X_test)
    metrics[model] = calculate_metrics(y_test, y_pred)
    
    # Store the performance
    df_performance = pd.concat([df_performance, df_model_performance], ignore_index=True)

In [ ]:
df_metrics = pd.DataFrame(metrics).T
df_metrics.reset_index(inplace=True)
df_metrics = df_metrics.rename(columns = {'index':'Model'})
print(df_metrics)

In [ ]:
print(df_performance)

In [ ]:
df_results = []
df_results = pd.merge(df_performance, df_metrics)
df_results

In [ ]:
def barplot_metrics(df_results):
    # Visualize bar plots for all metrics
    cmap = plt.get_cmap('Set2')
    colors = [cmap(i) for i in range(len(df_results.columns))]    
    ax = df_results.loc[:, df_results.columns != "Train Time"].plot.bar(rot=0, color = colors)

    # Set labels
    plt.xlabel('Models')
    plt.ylabel('Accuracy Scores')
    bar_positions = np.arange(len(df_results))
    plt.xticks(bar_positions, df_results["Model"], rotation=90)
    plt.title('Comparison of Model Performance')

    # Visualize the 'Train Time' on top of the 'Train Accuracy' bar
    for idx, value in enumerate(df_results['Train Time']):
        ax.text(idx - 0.15, df_results['Train Accuracy'][idx] + 0.015, f'[{value:.2f}sec]', fontsize=9)

    # Display the legend outside the plot area
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.2))

    # Display the plot
    plt.show()

In [ ]:
barplot_metrics(df_results)

## Data Modeling III - Anomaly detection

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from warnings import filterwarnings 
filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import mean_squared_error

In [ ]:
from statsmodels.tsa import arima_process

In [ ]:
process = arima_process.ArmaProcess(ar = [.9, -.3], ma = [2])

In [ ]:
col_name = 'VehicleSpeed'
df_DUT = df1[[col_name]]
#df_DUT = df_DUT.set_index(pd.to_datetime(df_DUT.index)) \
#    .drop(columns=['Unnamed: 0','Date']) \
#    .rename('values')
df_DUT.head()

In [ ]:
plt.plot(df_DUT)
plt.title('Vehicle Speed', loc = 'left')
plt.grid();
### Build historical dataset
y_hist = df_DUT[:7000]
y_future = df_DUT[7001:]

print('Historical:')
print(y_hist.tail())
print('=========\nFuture:')
print(y_future.head())
plt.plot(y_hist, label = 'historical')
plt.plot(y_future, label = 'future')
plt.legend();

For our exercise, we will use a built in estimator from statsmodels -- the `STL` model.  To use this model, create an instance of the `STL` estimator and pass `y_hist` and a period value of 12.  

Use the `stl` instance to fit the model, assigning the fit results to `results` below.  This results object will contain the trend as an attribute.  Uncomment the code to see the trend plotted with the original data after fitting.

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.filters.filtertools import convolution_filter
import statsmodels.graphics.tsaplots as tsplots
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.tsa.forecasting.stl import STLForecast
from statsmodels.tsa.arima.model import ARIMA
stl = STL(y_hist, period = 20)
results = stl.fit()
plt.plot(results.trend)
plt.plot(y_hist)
plt.title('Trend with CO2 Data')

In [ ]:
season_and_trend = results.seasonal + results.trend
### END SOLUTION

## Answer Check
plt.plot(season_and_trend, label = 'seasonal + trend')
plt.plot(y_hist, label = 'actual')
plt.grid()
plt.legend()
plt.xticks(rotation = 90);

### ARMA Process

### Extract the trend

In [ ]:
from statsmodels.tsa.filters.filtertools import convolution_filter
import statsmodels.graphics.tsaplots as tsaplots
from statsmodels.tsa.seasonal import _extrapolate_trend

In [ ]:
y_hist.info()

In [ ]:
period = 1000


filt = np.ones(period+1)
filt[0] = 0.5
filt[-1] = 0.5
filt /= period

sum(filt)

In [ ]:
trend = convolution_filter(y_hist, filt)
trend = _extrapolate_trend(trend, period + 1)